### Final Results With Comparison to Different Model Performance

In [1]:
!pip install xgboost


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
print(pd.__version__)
import numpy as np
import json
import logging
import xgboost as xgb
import ast
import itertools
import scipy.stats as ss
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2
import os
import sys

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

2.1.3


In [3]:
sys.path.append('/root/Unified_LN_V2/')
from Utilities import *

In [4]:
def air_calc_new(pred,prefix):
    air = {}
    target_control_dict = [
            {'age_cuts': {
                'target': ['Age 62 and over'],
                'control': ['Age below 62']
            }},
            {'Race': {
                'target': ['black', 'hispanic'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['black'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['hispanic'],
                'control': ['white']
            }},
            {'Gender': {
                'target': ['F'],
                'control': ['M']
            }},
            # 'mmct_tag': {
            #     'target': ['mmct'],
            #     'control': ['non-mmct']
            # }
    ]

    for var in target_control_dict:
        for i in var:
            target_list = var[i]['target']
            control_list = var[i]['control']
            p_target_list= ','.join(target_list) if (
                (len(target_list)<= 1) and (('black' in target_list) or ('hispanic' in target_list))
            ) else ''
            
            ttd_target= pred[pred[i].isin(target_list)]['con_wt'].sum()
            approved_target= pred[pred[i].isin(target_list)]['approved'].multiply(pred[pred[i].isin(target_list)]['con_wt']).sum()
            ttd_control= pred[pred[i].isin(control_list)]['con_wt'].sum()
            approved_control= pred[pred[i].isin(control_list)]['approved'].multiply(pred[pred[i].isin(control_list)]['con_wt']).sum()
            app_pct_target= round(approved_target/ttd_target, 4)
            app_pct_control= round(approved_control/ttd_control, 4)
            air[prefix+i+p_target_list]= [round(app_pct_target/app_pct_control, 4)]
        
            print({'variable': i, 'target_list': p_target_list, 'ttd_target': ttd_target, 'ttd_control': ttd_control, 
                   'approved_target': approved_target, 'approved_control': approved_control})
            
    # print(air)
        
    return air

In [5]:
col_tuple_pass= ["unique_id", 'client', "wt2", 'fico_pb', 'auto_score', 'approve_flag', 'funded_flag', 'bad_flag', 'Race', 'Gender', 'age_cuts']

In [6]:
def get_risk_rank(data, target, cols_tuple, score_col= 'p1', weight= 'RECWGT', asc= False):
    dd = data.loc[:, cols_tuple + [score_col]]
    # dd = pd.DataFrame(dd)
    # dd = dd.reset_index()
    dd['RECWGT'] = dd["wt2"]
    dd['conwt'] = 1
    dd[target] = dd[target] 
    # pred = pd.concat([dd.loc[:, cols_tuple + ['RECWGT', 'conwt']],p],axis=1)
    pred= dd.copy()
    # display(pred.head())
    print(pred[weight].sum(),((pred[weight]*pred[target]).sum()),pred[weight].max())

    data_ks_cap5= rnkOrderingN(pred, target, weight, score_col, 5, asc)
    # display(data_ks_cap5[0])
    # print(data_ks_cap5[1])
    # data_ks_cap10= rnkOrderingN(pred, target, weight, score_col, 10, asc)
    # display(data_ks_cap10[0])
    # print(data_ks_cap10[1])
    return data_ks_cap5

In [7]:
def get_segment(d):
    if (d['derogcount']> 0):
        return 'Derog'
    elif ((d['derogcount']== 0) and (d['assetprop']> 0)):
        return 'Property Owner'
    elif ((d['derogcount']== 0) and (d['assetprop']== 0)):
        return 'Non Property Owner'
    else:
        return 'Missing'

In [8]:
def Equal_Approval_Cutoff_Barsath(pred, target, score_col, asc= True):
    pred= pred.copy()
    actual_approved_count= pred[target].sum()
    pred= pred.sort_values(by= [score_col], ascending= asc)
    pred['con_wt']= 1
    pred['cumsum']= pred['con_wt'].cumsum()
    pred['approved']= np.where(pred['cumsum']<= actual_approved_count, 1, 0)
    return pred #cutoff, original_dataset

## Reading Scored Data - Overall LDA

In [10]:
path_overall= '/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/With_LDA/Iteration_3/' #/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/With_LDA/
os.listdir(path_overall)

['With_LDA_Train_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 '050424_LN_Auto_KGB_LDA_No_Segment_Hopt_4.csv',
 'With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'xgbm_ln_auto_v2_hpt_best_with_lda_3.zip',
 'With_LDA_Test_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-04-08.csv']

In [11]:
# train_scored_all= pd.read_csv(f"{path_overall}With_LDA_Train_Auto_LN_V2_Scored_Data_2024-03-29.csv")
# test_scored_all= pd.read_csv(f"{path_overall}With_LDA_Test_Auto_LN_V2_Scored_Data_2024-03-29.csv")
# oot_scored_all= pd.read_csv(f"{path_overall}With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-03-29.csv")
# ttd_scored_all= pd.read_csv(f"{path_overall}With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-03-29.csv")

# train_scored_all= pd.read_csv(f"{path_overall}With_LDA_Train_Auto_LN_V2_Scored_Data_2024-04-02.csv")
# test_scored_all= pd.read_csv(f"{path_overall}With_LDA_Test_Auto_LN_V2_Scored_Data_2024-04-02.csv")
# oot_scored_all= pd.read_csv(f"{path_overall}With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-04-02.csv")
# ttd_scored_all= pd.read_csv(f"{path_overall}With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-04-02.csv")

train_scored_all= pd.read_csv(f"{path_overall}With_LDA_Train_Auto_LN_V2_Scored_Data_2024-04-08.csv")
test_scored_all= pd.read_csv(f"{path_overall}With_LDA_Test_Auto_LN_V2_Scored_Data_2024-04-08.csv")
oot_scored_all= pd.read_csv(f"{path_overall}With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-04-08.csv")
ttd_scored_all= pd.read_csv(f"{path_overall}With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-04-08.csv")

train_scored_all.shape, test_scored_all.shape, oot_scored_all.shape, ttd_scored_all.shape

((146855, 12), (62938, 12), (37121, 12), (685215, 12))

In [12]:
funded_scored_all= pd.concat([train_scored_all, test_scored_all, oot_scored_all]).reset_index(drop= True)
funded_scored_all.shape, ttd_scored_all.shape, ttd_scored_all['funded_flag'].sum()

((246914, 12), (685215, 12), 200148.0)

In [13]:
funded_scored_all['unique_id']= funded_scored_all['unique_id'].astype(str)
ttd_scored_all['unique_id']= ttd_scored_all['unique_id'].astype(str)

In [14]:
funded_scored_all.rename(columns= {'p1': 'p1_overall_lda'}, inplace= True)
ttd_scored_all.rename(columns= {'p1': 'p1_overall_lda'}, inplace= True)

### Reading V 1.5 Scored Data

In [15]:
v15= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/040424_Auto_LN_V1_5_Scored_Data.csv')
v15.head()

,unique_id,client,p1_v1_5,fico_pb,dataset
0,453208,SECU,0.053346,624.0,fico_lt_660
1,453394,SECU,0.064319,0.0,fico_lt_660
2,453515,SECU,0.086988,614.0,fico_lt_660
3,453555,SECU,0.035332,637.0,fico_lt_660
4,453594,SECU,0.094630,587.0,fico_lt_660


In [16]:
v15.shape, ttd_scored_all.shape, funded_scored_all.shape

((824212, 5), (685215, 12), (246914, 12))

In [17]:
v15['unique_id']= v15['unique_id'].astype(str)

In [18]:
print(ttd_scored_all.shape)
ttd_scored_all= ttd_scored_all.merge(v15[['unique_id', 'client', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
ttd_scored_all.shape

(685215, 12)


(685215, 13)

In [19]:
ttd_scored_all['p1_v1_5'].isnull().sum()

0

In [20]:
ttd_scored_all[['p1_v1_5', 'p1_overall_lda']].describe()

,p1_v1_5,p1_overall_lda
count,685215.000000,685215.000000
mean,0.038407,0.021863
std,0.032355,0.022076
min,0.007456,0.005289
25%,0.014268,0.007273
50%,0.025966,0.015229
75%,0.052640,0.024805
max,0.190158,0.114281


In [21]:
print(funded_scored_all.shape)
funded_scored_all= funded_scored_all.merge(v15[['unique_id', 'client', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
funded_scored_all.shape, funded_scored_all['p1_v1_5'].isnull().sum()

(246914, 12)


((246914, 13), 0)

In [22]:
# rnkOrderingN(funded_scored_all[~(funded_scored_all['fico_pb']>= 740)], 'bad_flag', None, 'p1_overall_lda', 5)

In [23]:
# rnkOrderingN(funded_scored_all[~(funded_scored_all['fico_pb']>= 740)], 'bad_flag', None, 'p1_v1_5', 5)

#### Reading Derog Dataset

In [24]:
path_owner= '/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Derog_Model/'
os.listdir(path_owner)

['Derog_290324_LN_Auto_KGB_LDA_Hopt_1.csv',
 'Mono_Data',
 'Data',
 'Derog_Test_Auto_LN_V2_Scored_Data_2024-03-29.csv',
 'Derog_TTD_Auto_LN_V2_Scored_Data_2024-03-29.csv',
 'Derog_Oot_Auto_LN_V2_Scored_Data_2024-03-29.csv',
 'Derog_Train_Auto_LN_V2_Scored_Data_2024-03-29.csv',
 'xgbm_ln_auto_v2_hpt_best_1_Derog.zip']

In [25]:
train_scored_owner= pd.read_csv(f'{path_owner}Derog_Train_Auto_LN_V2_Scored_Data_2024-03-29.csv')
test_scored_owner= pd.read_csv(f'{path_owner}Derog_Test_Auto_LN_V2_Scored_Data_2024-03-29.csv')
oot_scored_owner= pd.read_csv(f'{path_owner}Derog_Oot_Auto_LN_V2_Scored_Data_2024-03-29.csv')
ttd_scored_owner= pd.read_csv(f'{path_owner}Derog_TTD_Auto_LN_V2_Scored_Data_2024-03-29.csv')
train_scored_owner.shape, test_scored_owner.shape, oot_scored_owner.shape, ttd_scored_owner.shape

((15871, 12), (6803, 12), (3911, 12), (107013, 12))

In [26]:
train_scored_owner['unique_id']= train_scored_owner['unique_id'].astype(str)
test_scored_owner['unique_id']= test_scored_owner['unique_id'].astype(str)
oot_scored_owner['unique_id']= oot_scored_owner['unique_id'].astype(str)
ttd_scored_owner['unique_id']= ttd_scored_owner['unique_id'].astype(str)

In [27]:
train_scored_owner['dataset']= 'Derog'
test_scored_owner['dataset']= 'Derog'
oot_scored_owner['dataset']= 'Derog'
ttd_scored_owner['dataset']= 'Derog'

#### Reading Non Derog Dataset

In [28]:
path_non_owner= '/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Non_Derog_Model/'
os.listdir(path_non_owner)

['Non_Derog_Train_Auto_LN_V2_Scored_Data_2024-04-01.csv',
 'Non_Derog_Oot_Auto_LN_V2_Scored_Data_2024-04-01.csv',
 'Mono_Data',
 'Non_Derog_Test_Auto_LN_V2_Scored_Data_2024-04-01.csv',
 'Data',
 'Non_Derog_TTD_Auto_LN_V2_Scored_Data_2024-04-01.csv',
 'Non_Derog_290324_LN_Auto_KGB_LDA_Hopt_1.csv',
 'xgbm_ln_auto_v2_hpt_best_1_Non_Derog.zip']

In [29]:
train_scored_nonowner= pd.read_csv(f'{path_non_owner}Non_Derog_Train_Auto_LN_V2_Scored_Data_2024-04-01.csv')
test_scored_nonowner= pd.read_csv(f'{path_non_owner}Non_Derog_Test_Auto_LN_V2_Scored_Data_2024-04-01.csv')
oot_scored_nonowner= pd.read_csv(f'{path_non_owner}Non_Derog_Oot_Auto_LN_V2_Scored_Data_2024-04-01.csv')
ttd_scored_nonowner= pd.read_csv(f'{path_non_owner}Non_Derog_TTD_Auto_LN_V2_Scored_Data_2024-04-01.csv')
train_scored_nonowner.shape, test_scored_nonowner.shape, oot_scored_nonowner.shape, ttd_scored_nonowner.shape

((130983, 12), (56136, 12), (33210, 12), (578202, 12))

In [30]:
train_scored_nonowner['unique_id']= train_scored_nonowner['unique_id'].astype(str)
test_scored_nonowner['unique_id']= test_scored_nonowner['unique_id'].astype(str)
oot_scored_nonowner['unique_id']= oot_scored_nonowner['unique_id'].astype(str)
ttd_scored_nonowner['unique_id']= ttd_scored_nonowner['unique_id'].astype(str)

In [31]:
train_scored_nonowner['dataset']= 'Non Derog'
test_scored_nonowner['dataset']= 'Non Derog'
oot_scored_nonowner['dataset']= 'Non Derog'
ttd_scored_nonowner['dataset']= 'Non Derog'

In [32]:
train_ono= pd.concat([train_scored_owner, train_scored_nonowner]).reset_index(drop= True)
test_ono= pd.concat([test_scored_owner, test_scored_nonowner]).reset_index(drop= True)
oot_ono= pd.concat([oot_scored_owner, oot_scored_nonowner]).reset_index(drop= True)
ttd_ono= pd.concat([ttd_scored_owner, ttd_scored_nonowner]).reset_index(drop= True)
train_ono.shape, test_ono.shape, oot_ono.shape, ttd_ono.shape

((146854, 13), (62939, 13), (37121, 13), (685215, 13))

In [33]:
train_scored_all.shape, test_scored_all.shape, oot_scored_all.shape, ttd_scored_all.shape

((146855, 12), (62938, 12), (37121, 12), (685215, 13))

In [34]:
print(train_ono['dataset'].value_counts(dropna= False), train_scored_owner.shape, train_scored_nonowner.shape)
print(test_ono['dataset'].value_counts(dropna= False), test_scored_owner.shape, test_scored_nonowner.shape)
print(oot_ono['dataset'].value_counts(dropna= False), oot_scored_owner.shape, oot_scored_nonowner.shape)
print(ttd_ono['dataset'].value_counts(dropna= False), ttd_scored_owner.shape, ttd_scored_nonowner.shape)

dataset
Non Derog    130983
Derog         15871
Name: count, dtype: int64 (15871, 13) (130983, 13)
dataset
Non Derog    56136
Derog         6803
Name: count, dtype: int64 (6803, 13) (56136, 13)
dataset
Non Derog    33210
Derog         3911
Name: count, dtype: int64 (3911, 13) (33210, 13)
dataset
Non Derog    578202
Derog        107013
Name: count, dtype: int64 (107013, 13) (578202, 13)


In [35]:
# del train_scored_all, train_scored_nonowner, train_scored_owner
# del test_scored_all, test_scored_nonowner, test_scored_owner
# del oot_scored_all, oot_scored_nonowner, oot_scored_owner
# del ttd_scored_nonowner, ttd_scored_owner

In [36]:
import gc
gc.collect()

0

### Merging p1-overall

In [37]:
print(train_ono.shape)
train_ono= train_ono.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(train_ono.shape)
print(train_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

print(test_ono.shape)
test_ono= test_ono.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(test_ono.shape)
print(test_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

print(oot_ono.shape)
oot_ono= oot_ono.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(oot_ono.shape)
print(oot_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

print(ttd_ono.shape)
ttd_ono= ttd_ono.merge(ttd_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(ttd_ono.shape)
print(ttd_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

(146854, 13)
(146854, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64
(62939, 13)
(62939, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64
(37121, 13)
(37121, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64
(685215, 13)
(685215, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64


In [38]:
ttd_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe(), train_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe(), test_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe(), oot_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe()

(                  p1  p1_overall_lda        p1_v1_5
 count  685215.000000   685215.000000  685215.000000
 mean        0.021729        0.021863       0.038407
 std         0.018605        0.022076       0.032355
 min         0.005160        0.005289       0.007456
 25%         0.008211        0.007273       0.014268
 50%         0.015675        0.015229       0.025966
 75%         0.028204        0.024805       0.052640
 max         0.172651        0.114281       0.190158,
                   p1  p1_overall_lda        p1_v1_5
 count  146854.000000   146854.000000  146854.000000
 mean        0.017096        0.016758       0.026971
 std         0.015128        0.017344       0.024389
 min         0.005160        0.005289       0.007456
 25%         0.006272        0.006565       0.012016
 50%         0.012448        0.009939       0.017433
 75%         0.020681        0.020090       0.031214
 max         0.172651        0.114281       0.188945,
                  p1  p1_overall_lda       p

In [39]:
train_ono.groupby(['dataset'])[['p1', 'p1_overall_lda', 'p1_v1_5']].apply(lambda x: x.describe()).T

dataset           Derog                                                    \
                  count      mean       std       min       25%       50%   
p1              15871.0  0.030910  0.022180  0.010060  0.015102  0.022748   
p1_overall_lda  15871.0  0.021383  0.020954  0.005289  0.007477  0.015029   
p1_v1_5         15871.0  0.039148  0.029790  0.007456  0.016645  0.029073   

dataset                            Non Derog                                \
                     75%       max     count      mean       std       min   
p1              0.039300  0.172651  130983.0  0.015423  0.013078  0.005160   
p1_overall_lda  0.024566  0.114281  130983.0  0.016198  0.016767  0.005289   
p1_v1_5         0.052276  0.182189  130983.0  0.025496  0.023222  0.007533   

dataset                                                 
                     25%       50%       75%       max  
p1              0.006048  0.011072  0.018831  0.080271  
p1_overall_lda  0.006554  0.009230  0.019801  0.114281  
p1_v1_5         0.011710  0.016667  0.028560  0.188945

In [40]:
train_ono[train_ono['dataset'].isin(['Property Owner'])].shape

(0, 15)

In [41]:
train_ono['dataset'].value_counts(dropna= False)

dataset
Non Derog    130983
Derog         15871
Name: count, dtype: int64

In [42]:
def get_model_comparison_segment(train_d, test_d, oot_d, ttd_d, segment, pref, d):
    train_d= train_d.copy()
    test_d= test_d.copy()
    oot_d= oot_d.copy()
    ttd_d= ttd_d.copy()
    print({'segment': segment})#, 'overall shape': data.shape, 'segmented shape': data[data['dataset'].isin(segment)].shape})
    
    a= {'Model': pref, 'Dataset': d}
    
    a[f'KS Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Oot Model']= rnkOrderingN(oot_d[oot_d['dataset'].isin(segment)], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Oot FICO']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS']
    a[f'KS Non Weighted Oot Auto Score']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS']
    
    a['Bad Rate Train']= train_d[train_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Test']= test_d[test_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Oot']= oot_d[oot_d['dataset'].isin(segment)]['bad_flag'].mean()
    
    air_sgement_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'p1'), 'AIR TTD')
    a[f'AIR Age Model']= air_sgement_model['AIR TTDage_cuts']
    a[f'AIR Race Model']= air_sgement_model['AIR TTDRace']
    a[f'AIR Race Black Model']= air_sgement_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic Model']= air_sgement_model['AIR TTDRacehispanic']
    a[f'AIR Gender Model']= air_sgement_model['AIR TTDGender']
    
    air_fico_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'fico_pb', asc= False), 'AIR TTD')
    a[f'AIR Age FICO']= air_fico_model['AIR TTDage_cuts']
    a[f'AIR Race FICO']= air_fico_model['AIR TTDRace']
    a[f'AIR Race Black FICO']= air_fico_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic FICO']= air_fico_model['AIR TTDRacehispanic']
    a[f'AIR Gender FICO']= air_fico_model['AIR TTDGender']
    
    return a

In [43]:
def get_model_comparison_overall(train_d, test_d, oot_d, ttd_d, segment, pref, d):
    train_d= train_d.copy()
    test_d= test_d.copy()
    oot_d= oot_d.copy()
    ttd_d= ttd_d.copy()
    print({'segment': segment})#, 'overall shape': data.shape, 'segmented shape': data[data['dataset'].isin(segment)].shape})
    
    a= {'Model': pref, 'Dataset': d}
    
    a[f'KS Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Oot Model']= rnkOrderingN(oot_d[oot_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Oot FICO']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS']
    a[f'KS Non Weighted Oot Auto Score']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS']
    
    a['Bad Rate Train']= train_d[train_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Test']= test_d[test_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Oot']= oot_d[oot_d['dataset'].isin(segment)]['bad_flag'].mean()
    
    air_sgement_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'p1_overall_lda'), 'AIR TTD')
    a[f'AIR Age Model']= air_sgement_model['AIR TTDage_cuts']
    a[f'AIR Race Model']= air_sgement_model['AIR TTDRace']
    a[f'AIR Race Black Model']= air_sgement_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic Model']= air_sgement_model['AIR TTDRacehispanic']
    a[f'AIR Gender Model']= air_sgement_model['AIR TTDGender']
    
    air_fico_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'fico_pb', asc= False), 'AIR TTD')
    a[f'AIR Age FICO']= air_fico_model['AIR TTDage_cuts']
    a[f'AIR Race FICO']= air_fico_model['AIR TTDRace']
    a[f'AIR Race Black FICO']= air_fico_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic FICO']= air_fico_model['AIR TTDRacehispanic']
    a[f'AIR Gender FICO']= air_fico_model['AIR TTDGender']
    
    return a
    

In [44]:
def get_model_comparison_v15(train_d, test_d, oot_d, ttd_d, segment, pref, d):
    train_d= train_d.copy()
    test_d= test_d.copy()
    oot_d= oot_d.copy()
    ttd_d= ttd_d.copy()
    print({'segment': segment})#, 'overall shape': data.shape, 'segmented shape': data[data['dataset'].isin(segment)].shape})
    
    a= {'Model': pref, 'Dataset': d}
    
    a[f'KS Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Oot Model']= rnkOrderingN(oot_d[oot_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Oot FICO']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS']
    a[f'KS Non Weighted Oot Auto Score']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS']
    
    a['Bad Rate Train']= train_d[train_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Test']= test_d[test_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Oot']= oot_d[oot_d['dataset'].isin(segment)]['bad_flag'].mean()
    
    air_sgement_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'p1_v1_5'), 'AIR TTD')
    a[f'AIR Age Model']= air_sgement_model['AIR TTDage_cuts']
    a[f'AIR Race Model']= air_sgement_model['AIR TTDRace']
    a[f'AIR Race Black Model']= air_sgement_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic Model']= air_sgement_model['AIR TTDRacehispanic']
    a[f'AIR Gender Model']= air_sgement_model['AIR TTDGender']
    
    air_fico_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'fico_pb', asc= False), 'AIR TTD')
    a[f'AIR Age FICO']= air_fico_model['AIR TTDage_cuts']
    a[f'AIR Race FICO']= air_fico_model['AIR TTDRace']
    a[f'AIR Race Black FICO']= air_fico_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic FICO']= air_fico_model['AIR TTDRacehispanic']
    a[f'AIR Gender FICO']= air_fico_model['AIR TTDGender']
    
    return a

In [45]:
pd.concat(
    [pd.DataFrame(get_model_comparison_segment(train_ono, test_ono, oot_ono, ttd_ono, ['Derog'], 'Derog', 'Derog')),
     pd.DataFrame(get_model_comparison_overall(train_ono, test_ono, oot_ono, ttd_ono, ['Derog'], 'Overall', 'Derog')),
     pd.DataFrame(get_model_comparison_v15(train_ono, test_ono, oot_ono, ttd_ono, ['Derog'], 'V1.5', 'Derog'))]
)

{'segment': ['Derog']}
{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 9123, 'ttd_control': 97433, 'approved_target': 5730, 'approved_control': 39553}
{'variable': 'Race', 'target_list': '', 'ttd_target': 20954, 'ttd_control': 58555, 'approved_target': 7261, 'approved_control': 27425}
{'variable': 'Race', 'target_list': 'black', 'ttd_target': 12925, 'ttd_control': 58555, 'approved_target': 4326, 'approved_control': 27425}
{'variable': 'Race', 'target_list': 'hispanic', 'ttd_target': 8029, 'ttd_control': 58555, 'approved_target': 2935, 'approved_control': 27425}
{'variable': 'Gender', 'target_list': '', 'ttd_target': 40718, 'ttd_control': 58619, 'approved_target': 16577, 'approved_control': 26202}
{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 9123, 'ttd_control': 97433, 'approved_target': 4813, 'approved_control': 40586}
{'variable': 'Race', 'target_list': '', 'ttd_target': 20954, 'ttd_control': 58555, 'approved_target': 7366, 'approved_control': 27414}
{'variable'

,Model,Dataset,KS Weighted Train Model,KS Weighted Test Model,KS Non Weighted Train Model,KS Non Weighted Test Model,KS Non Weighted Oot Model,KS Non Weighted Oot FICO,KS Non Weighted Oot Auto Score,Bad Rate Train,Bad Rate Test,Bad Rate Oot,AIR Age Model,AIR Race Model,AIR Race Black Model,AIR Race Hispanic Model,AIR Gender Model,AIR Age FICO,AIR Race FICO,AIR Race Black FICO,AIR Race Hispanic FICO,AIR Gender FICO
0,Derog,Derog,35.40,32.96,34.28,30.97,27.77,35.76,29.93,0.028102,0.028517,0.029404,1.5470,0.7398,0.7146,0.7803,0.9107,1.2664,0.7507,0.5869,1.0145,0.8811
0,Overall,Derog,29.05,25.09,31.05,26.20,25.97,35.76,29.93,0.028102,0.028517,0.029404,1.4371,0.7863,0.7269,0.8821,0.9003,1.2664,0.7507,0.5869,1.0145,0.8811
0,V1.5,Derog,37.20,34.55,41.19,36.28,30.46,35.76,29.93,0.028102,0.028517,0.029404,1.4534,0.7481,0.6554,0.8973,0.9113,1.2664,0.7507,0.5869,1.0145,0.8811


In [46]:
pd.concat(
    [pd.DataFrame(get_model_comparison_segment(train_ono, test_ono, oot_ono, ttd_ono, ['Non Derog'], 'Non Derog', 'Non Derog')),
     pd.DataFrame(get_model_comparison_overall(train_ono, test_ono, oot_ono, ttd_ono, ['Non Derog'], 'Overall', 'Non Derog')),
     pd.DataFrame(get_model_comparison_v15(train_ono, test_ono, oot_ono, ttd_ono, ['Non Derog'], 'V1.5', 'Non Derog'))]
)

{'segment': ['Non Derog']}
{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 66248, 'ttd_control': 510285, 'approved_target': 56593, 'approved_control': 299054}
{'variable': 'Race', 'target_list': '', 'ttd_target': 95009, 'ttd_control': 327493, 'approved_target': 48074, 'approved_control': 219462}
{'variable': 'Race', 'target_list': 'black', 'ttd_target': 48009, 'ttd_control': 327493, 'approved_target': 24911, 'approved_control': 219462}
{'variable': 'Race', 'target_list': 'hispanic', 'ttd_target': 47000, 'ttd_control': 327493, 'approved_target': 23163, 'approved_control': 219462}
{'variable': 'Gender', 'target_list': '', 'ttd_target': 219543, 'ttd_control': 315294, 'approved_target': 131461, 'approved_control': 203833}
{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 66248, 'ttd_control': 510285, 'approved_target': 50534, 'approved_control': 305672}
{'variable': 'Race', 'target_list': '', 'ttd_target': 95009, 'ttd_control': 327493, 'approved_target': 47051, 'approved_

,Model,Dataset,KS Weighted Train Model,KS Weighted Test Model,KS Non Weighted Train Model,KS Non Weighted Test Model,KS Non Weighted Oot Model,KS Non Weighted Oot FICO,KS Non Weighted Oot Auto Score,Bad Rate Train,Bad Rate Test,Bad Rate Oot,AIR Age Model,AIR Race Model,AIR Race Black Model,AIR Race Hispanic Model,AIR Gender Model,AIR Age FICO,AIR Race FICO,AIR Race Black FICO,AIR Race Hispanic FICO,AIR Gender FICO
0,Non Derog,Non Derog,37.55,38.51,38.84,39.01,36.52,49.52,38.47,0.01362,0.013574,0.012948,1.4576,0.7551,0.7744,0.7354,0.9262,1.2735,0.7422,0.6419,0.8447,0.9288
0,Overall,Non Derog,38.40,39.25,38.84,39.27,39.35,49.52,38.47,0.01362,0.013574,0.012948,1.4218,0.7683,0.7743,0.7623,0.9270,1.2735,0.7422,0.6419,0.8447,0.9288
0,V1.5,Non Derog,43.42,51.40,44.28,48.90,43.58,49.52,38.47,0.01362,0.013574,0.012948,1.3940,0.7109,0.6767,0.7460,0.9375,1.2735,0.7422,0.6419,0.8447,0.9288


In [47]:
pd.concat(
    [pd.DataFrame(get_model_comparison_segment(train_ono, test_ono, oot_ono, ttd_ono, ['Non Derog', 'Derog'], 'Both', 'Both')),
     pd.DataFrame(get_model_comparison_overall(train_ono, test_ono, oot_ono, ttd_ono, ['Non Derog', 'Derog'], 'Overall', 'Both')),
     pd.DataFrame(get_model_comparison_v15(train_ono, test_ono, oot_ono, ttd_ono, ['Non Derog', 'Derog'], 'V1.5', 'Both'))]
)

{'segment': ['Non Derog', 'Derog']}
{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 75371, 'ttd_control': 607718, 'approved_target': 62181, 'approved_control': 338756}
{'variable': 'Race', 'target_list': '', 'ttd_target': 115963, 'ttd_control': 386048, 'approved_target': 55637, 'approved_control': 245760}
{'variable': 'Race', 'target_list': 'black', 'ttd_target': 60934, 'ttd_control': 386048, 'approved_target': 29177, 'approved_control': 245760}
{'variable': 'Race', 'target_list': 'hispanic', 'ttd_target': 55029, 'ttd_control': 386048, 'approved_target': 26460, 'approved_control': 245760}
{'variable': 'Gender', 'target_list': '', 'ttd_target': 260261, 'ttd_control': 373913, 'approved_target': 148524, 'approved_control': 229170}
{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 75371, 'ttd_control': 607718, 'approved_target': 55234, 'approved_control': 346371}
{'variable': 'Race', 'target_list': '', 'ttd_target': 115963, 'ttd_control': 386048, 'approved_target': 54450,

,Model,Dataset,KS Weighted Train Model,KS Weighted Test Model,KS Non Weighted Train Model,KS Non Weighted Test Model,KS Non Weighted Oot Model,KS Non Weighted Oot FICO,KS Non Weighted Oot Auto Score,Bad Rate Train,Bad Rate Test,Bad Rate Oot,AIR Age Model,AIR Race Model,AIR Race Black Model,AIR Race Hispanic Model,AIR Gender Model,AIR Age FICO,AIR Race FICO,AIR Race Black FICO,AIR Race Hispanic FICO,AIR Gender FICO
0,Both,Both,37.19,39.26,38.43,38.62,37.43,48.61,37.56,0.015185,0.015189,0.014682,1.4801,0.7537,0.7521,0.7553,0.9311,1.2856,0.7382,0.6168,0.8728,0.9247
0,Overall,Both,37.44,37.94,38.75,38.30,38.17,48.61,37.56,0.015185,0.015189,0.014682,1.4450,0.7644,0.7497,0.7804,0.9213,1.2856,0.7382,0.6168,0.8728,0.9247
0,V1.5,Both,44.17,46.02,44.58,48.09,44.14,48.61,37.56,0.015185,0.015189,0.014682,1.4141,0.7115,0.6549,0.7739,0.9355,1.2856,0.7382,0.6168,0.8728,0.9247


In [48]:
train_scored_all['dataset']= 'all'
test_scored_all['dataset']= 'all'
oot_scored_all['dataset']= 'all'
ttd_scored_all['dataset']= 'all'

In [49]:
print(train_scored_all.shape)
train_scored_all= train_scored_all.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(train_scored_all.shape)
print(train_scored_all[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

print(test_scored_all.shape)
test_scored_all= test_scored_all.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(test_scored_all.shape)
print(test_scored_all[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

print(oot_scored_all.shape)
oot_scored_all= oot_scored_all.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(oot_scored_all.shape)
print(oot_scored_all[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

(146855, 13)
(146855, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64
(62938, 13)
(62938, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64
(37121, 13)
(37121, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64


In [50]:
pd.concat(
    [
     pd.DataFrame(get_model_comparison_overall(train_scored_all, test_scored_all, oot_scored_all, ttd_scored_all, ['all'], 'Overall', 'all')),
     pd.DataFrame(get_model_comparison_v15(train_scored_all, test_scored_all, oot_scored_all, ttd_scored_all, ['all'], 'V1.5', 'all'))]
)

{'segment': ['all']}
{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 75371, 'ttd_control': 607718, 'approved_target': 60916, 'approved_control': 339920}
{'variable': 'Race', 'target_list': '', 'ttd_target': 115963, 'ttd_control': 386048, 'approved_target': 56342, 'approved_control': 245420}
{'variable': 'Race', 'target_list': 'black', 'ttd_target': 60934, 'ttd_control': 386048, 'approved_target': 29044, 'approved_control': 245420}
{'variable': 'Race', 'target_list': 'hispanic', 'ttd_target': 55029, 'ttd_control': 386048, 'approved_target': 27298, 'approved_control': 245420}
{'variable': 'Gender', 'target_list': '', 'ttd_target': 260261, 'ttd_control': 373913, 'approved_target': 147548, 'approved_control': 230045}
{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 75371, 'ttd_control': 607718, 'approved_target': 55237, 'approved_control': 346368}
{'variable': 'Race', 'target_list': '', 'ttd_target': 115963, 'ttd_control': 386048, 'approved_target': 54451, 'approved_cont

,Model,Dataset,KS Weighted Train Model,KS Weighted Test Model,KS Non Weighted Train Model,KS Non Weighted Test Model,KS Non Weighted Oot Model,KS Non Weighted Oot FICO,KS Non Weighted Oot Auto Score,Bad Rate Train,Bad Rate Test,Bad Rate Oot,AIR Age Model,AIR Race Model,AIR Race Black Model,AIR Race Hispanic Model,AIR Gender Model,AIR Age FICO,AIR Race FICO,AIR Race Black FICO,AIR Race Hispanic FICO,AIR Gender FICO
0,Overall,all,37.42,39.07,38.63,38.06,38.74,44.62,39.01,0.015165,0.015237,0.014682,1.4450,0.7644,0.7497,0.7804,0.9215,1.286,0.7384,0.6168,0.8728,0.9247
0,V1.5,all,44.74,44.22,45.67,45.13,44.14,44.62,39.01,0.015165,0.015237,0.014682,1.4141,0.7115,0.6549,0.7740,0.9355,1.286,0.7384,0.6168,0.8728,0.9247


### AIR Continum

In [51]:
ttd_scored_all.describe()

,wt2,fico_pb,auto_score,approve_flag,funded_flag,bad_flag,p1_overall_lda,p1_v1_5
count,685215.0,678989.000000,685215.000000,685215.000000,401605.000000,200148.000000,685215.000000,685215.000000
mean,1.0,663.072047,716.564983,0.586101,0.498370,0.015224,0.021863,0.038407
std,0.0,165.346145,45.649687,0.492531,0.499998,0.122442,0.022076,0.032355
min,1.0,0.000000,548.000000,0.000000,0.000000,0.000000,0.005289,0.007456
25%,1.0,627.000000,686.000000,0.000000,0.000000,0.000000,0.007273,0.014268
50%,1.0,688.000000,715.000000,1.000000,0.000000,0.000000,0.015229,0.025966
75%,1.0,752.000000,748.000000,1.000000,1.000000,0.000000,0.024805,0.052640
max,1.0,9999.000000,827.000000,1.000000,1.000000,1.000000,0.114281,0.190158


In [52]:
## Creating 20-Decile
def get_data_cuts(data, q, score, asc= True):
    data= data.copy()
    data= data.sort_values(by= [score], ascending= asc).reset_index(drop= True)
    data['counter']= 1
    data['cumsum']= data['counter'].cumsum()
    
    bins= [-np.inf] + [int(round((len(data)/q)*i)) for i in range(1, q)] + [np.inf]
    labels= [i for i in range(1, q+1)]
    print(bins)
    print(labels)
    
    data['decile']= pd.cut(x= data['cumsum'], bins= bins, labels= labels)
    
    return data

In [53]:
ttd_scored_all_lda= get_data_cuts(ttd_scored_all, q= 20, score= 'p1_overall_lda')
ttd_scored_all.shape, ttd_scored_all_lda.shape

[-inf, 34261, 68522, 102782, 137043, 171304, 205564, 239825, 274086, 308347, 342608, 376868, 411129, 445390, 479650, 513911, 548172, 582433, 616694, 650954, inf]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


((685215, 14), (685215, 17))

In [54]:
ttd_scored_all_lda['decile'].value_counts(dropna= False).sort_index()

decile
1     34261
2     34261
3     34260
4     34261
5     34261
6     34260
7     34261
8     34261
9     34261
10    34261
11    34260
12    34261
13    34261
14    34260
15    34261
16    34261
17    34261
18    34261
19    34260
20    34261
Name: count, dtype: int64

In [55]:
%%time
air_cont_lda= []

for d in sorted(list(ttd_scored_all_lda['decile'].unique())):
    a= {}
    temp= ttd_scored_all_lda[ttd_scored_all_lda['decile']== d]
    print(d, temp.shape)
    a['decile']= d
    a['apps']= len(temp)
    a['min_pd']= temp['p1_overall_lda'].min()
    a['mean_pd']= temp['p1_overall_lda'].mean()
    a['max_pd']= temp['p1_overall_lda'].max()
    
    air_temp= air_calc_new(Equal_Approval_Cutoff_Barsath(temp, 'approve_flag', 'p1_overall_lda'), 'AIR TTD')
    # print(air_temp)
    a[f'AIR Age']= air_temp['AIR TTDage_cuts'][0]
    a[f'AIR Race']= air_temp['AIR TTDRace'][0]
    a[f'AIR Race Black']= air_temp['AIR TTDRaceblack'][0]
    a[f'AIR Race Hispanic']= air_temp['AIR TTDRacehispanic'][0]
    a[f'AIR Gender']= air_temp['AIR TTDGender'][0]
    
    air_cont_lda.append(a)
    print(a)
    del a

1 (34261, 17)
{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 5432, 'ttd_control': 28829, 'approved_target': 4278, 'approved_control': 22631}
{'variable': 'Race', 'target_list': '', 'ttd_target': 3227, 'ttd_control': 23037, 'approved_target': 2562, 'approved_control': 18086}
{'variable': 'Race', 'target_list': 'black', 'ttd_target': 1947, 'ttd_control': 23037, 'approved_target': 1541, 'approved_control': 18086}
{'variable': 'Race', 'target_list': 'hispanic', 'ttd_target': 1280, 'ttd_control': 23037, 'approved_target': 1021, 'approved_control': 18086}
{'variable': 'Gender', 'target_list': '', 'ttd_target': 11711, 'ttd_control': 20917, 'approved_target': 9167, 'approved_control': 16464}
{'decile': 1, 'apps': 34261, 'min_pd': 0.0052891406230628, 'mean_pd': 0.005289140623062798, 'max_pd': 0.0052891406230628, 'AIR Age': 1.0033, 'AIR Race': 1.0112, 'AIR Race Black': 1.0082, 'AIR Race Hispanic': 1.016, 'AIR Gender': 0.9945}
2 (34261, 17)
{'variable': 'age_cuts', 'target_list': '', '

In [56]:
air_cont_lda= pd.DataFrame(air_cont_lda)
air_cont_lda

,decile,apps,min_pd,mean_pd,max_pd,AIR Age,AIR Race,AIR Race Black,AIR Race Hispanic,AIR Gender
0,1,34261,0.005289,0.005289,0.005289,1.0033,1.0112,1.0082,1.0160,0.9945
1,2,34261,0.005289,0.005748,0.005875,0.8410,0.8744,0.9131,0.8107,0.9785
2,3,34260,0.005875,0.005959,0.006468,1.0391,1.1208,1.0960,1.1495,0.9777
3,4,34261,0.006468,0.006547,0.006586,0.8125,0.7487,0.7730,0.7160,0.9646
4,5,34261,0.006586,0.006971,0.007273,1.0727,1.0453,1.0253,1.0718,1.0267
5,6,34260,0.007273,0.007539,0.008012,1.1193,1.0408,0.9807,1.1171,1.0221
6,7,34261,0.008012,0.008366,0.008979,1.0519,0.9912,0.8765,1.0952,1.0106
7,8,34261,0.008979,0.009560,0.010209,1.1567,1.0255,1.0519,1.0042,1.0040
8,9,34261,0.010209,0.011897,0.013283,1.0813,0.9543,0.8952,1.0063,0.8812
9,10,34261,0.013283,0.014572,0.015229,1.0182,0.8862,0.9017,0.8732,1.0391


In [57]:
ttd_scored_all_v15= get_data_cuts(ttd_scored_all, q= 20, score= 'p1_v1_5')
print(ttd_scored_all.shape, ttd_scored_all_v15.shape)

air_cont_v15= []

for d in sorted(list(ttd_scored_all_v15['decile'].unique())):
    a= {}
    temp= ttd_scored_all_v15[ttd_scored_all_v15['decile']== d]
    print(d, temp.shape)
    a['decile']= d
    a['apps']= len(temp)
    a['min_pd']= temp['p1_v1_5'].min()
    a['mean_pd']= temp['p1_v1_5'].mean()
    a['max_pd']= temp['p1_v1_5'].max()
    
    air_temp= air_calc_new(Equal_Approval_Cutoff_Barsath(temp, 'approve_flag', 'p1_v1_5'), 'AIR TTD')
    # print(air_temp)
    a[f'AIR Age']= air_temp['AIR TTDage_cuts'][0]
    a[f'AIR Race']= air_temp['AIR TTDRace'][0]
    a[f'AIR Race Black']= air_temp['AIR TTDRaceblack'][0]
    a[f'AIR Race Hispanic']= air_temp['AIR TTDRacehispanic'][0]
    a[f'AIR Gender']= air_temp['AIR TTDGender'][0]
    
    air_cont_v15.append(a)
    print(a)
    del a, temp

[-inf, 34261, 68522, 102782, 137043, 171304, 205564, 239825, 274086, 308347, 342608, 376868, 411129, 445390, 479650, 513911, 548172, 582433, 616694, 650954, inf]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
(685215, 14) (685215, 17)
1 (34261, 17)
{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 5446, 'ttd_control': 28815, 'approved_target': 4825, 'approved_control': 24328}
{'variable': 'Race', 'target_list': '', 'ttd_target': 2564, 'ttd_control': 23780, 'approved_target': 2045, 'approved_control': 20399}
{'variable': 'Race', 'target_list': 'black', 'ttd_target': 1599, 'ttd_control': 23780, 'approved_target': 1319, 'approved_control': 20399}
{'variable': 'Race', 'target_list': 'hispanic', 'ttd_target': 965, 'ttd_control': 23780, 'approved_target': 726, 'approved_control': 20399}
{'variable': 'Gender', 'target_list': '', 'ttd_target': 11677, 'ttd_control': 20951, 'approved_target': 9875, 'approved_control': 17916}
{'decile': 1, 'apps': 34261, 'min_pd':

In [58]:
air_cont_v15= pd.DataFrame(air_cont_v15)
air_cont_v15

,decile,apps,min_pd,mean_pd,max_pd,AIR Age,AIR Race,AIR Race Black,AIR Race Hispanic,AIR Gender
0,1,34261,0.007456,0.008547,0.009295,1.0494,0.9298,0.9616,0.8770,0.9890
1,2,34261,0.009295,0.009922,0.010508,1.0131,0.9648,0.9900,0.9326,1.0111
2,3,34260,0.010508,0.011160,0.011649,1.0453,0.9808,1.0155,0.9418,0.9891
3,4,34261,0.011649,0.012351,0.013025,1.0264,0.9556,0.9664,0.9419,0.9984
4,5,34261,0.013025,0.013566,0.014268,0.9738,1.0756,1.0806,1.0709,1.0090
5,6,34260,0.014268,0.015005,0.015816,1.0544,1.0309,1.0266,1.0348,0.9812
6,7,34261,0.015816,0.016808,0.017901,1.0754,0.9790,0.9952,0.9642,1.0090
7,8,34261,0.017901,0.019051,0.020287,1.0748,0.9982,1.0121,0.9856,0.9995
8,9,34261,0.020287,0.021587,0.022776,1.0490,0.9609,0.9586,0.9628,0.9763
9,10,34261,0.022777,0.024364,0.025966,0.9204,0.9770,0.9757,0.9783,0.9881


In [51]:
ttd_scored_all.head()

,unique_id,client,wt2,fico_pb,auto_score,approve_flag,funded_flag,bad_flag,Race,Gender,age_cuts,p1_overall_lda,p1_v1_5,dataset
0,61929,Arkansas,1,726.0,707.0,1.0,0.0,NaN,white,M,Age below 62,0.009823,0.016805,all
1,TCI196142,Numerica,1,802.0,792.0,1.0,1.0,0.0,white,M,Age below 62,0.006650,0.009548,all
2,81484,Arkansas,1,649.0,766.0,0.0,NaN,NaN,white,M,Age 62 and over,0.005289,0.025465,all
3,518897,Tower,1,623.0,724.0,0.0,NaN,NaN,black,I,Age below 62,0.008783,0.045272,all
4,93535,Arkansas,1,673.0,705.0,1.0,0.0,NaN,white,F,Age below 62,0.026675,0.028135,all


In [54]:
rnkOrderingN(train_scored_all, 'bad_flag', 'wt2', 'p1_v1_5', 5)

(   min_score  max_score  mean_score         total         Goods         Bads  \
 0   0.050637   0.188945    0.079493  49036.601375  45764.793618  3271.807758   
 1   0.026892   0.050637    0.036356  49033.880827  47919.076067  1114.804760   
 2   0.017259   0.026892    0.021436  49033.372583  48537.475651   495.896932   
 3   0.012215   0.017259    0.014430  49034.536000  48778.201661   256.334340   
 4   0.007456   0.012215    0.010070  49034.263867  48937.865601    96.398266   
 
    Bad_Rate  Cum_Total  Cum_Good  Cum_Bad     KS  
 0      6.67       20.0     19.07    62.50  43.43  
 1      2.27       40.0     39.05    83.79  44.74  
 2      1.01       60.0     59.27    93.26  33.99  
 3      0.52       80.0     79.60    98.16  18.56  
 4      0.20      100.0    100.00   100.00   0.00  ,
 {'Gini': 56.28999999999999, 'KS': 44.74})

In [49]:
train_scored_all.shape, train_ono.shape

((146855, 13), (146854, 15))

In [ ]:
pd.concat(
    [pd.DataFrame(get_model_comparison_segment(['Non Derog'], 'Non Derog', 'Non Derog')),
     pd.DataFrame(get_model_comparison_overall(['Non Derog'], 'Overall', 'Non Derog')),
     pd.DataFrame(get_model_comparison_v15(['Non Derog'], 'V1.5', 'Non Derog'))]
)

### END